In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Mounting Google Drive storage to load required datasets

In order to read the required datasets for this analysis, `/content/drive` must be mounted using `drive.mount` into the notebook using the cell below.

In [ ]:
# Mount Google Drive storage to use dataset in the notebook

from google.colab import drive
drive.mount('/content/drive')

# COVID-19 Dataset

The datasets for this analysis were retrieved from the Philippine Department of Health's COVID-19 Data Drop. A number of datasets can be found from the website, which includes quarantine facility data from the entire Philippines. 

For this analysis, we are interested in the recorded information by the DOH regarding the cases of COVID-19, labelled as follows:

DOH COVID Data Drop_ 20220409 - 04 Case Information_batch_0<br>
DOH COVID Data Drop_ 20220409 - 04 Case Information_batch_1<br>
DOH COVID Data Drop_ 20220409 - 04 Case Information_batch_2<br>
DOH COVID Data Drop_ 20220409 - 04 Case Information_batch_3<br>

The link to the DOH COVID-19 Data Drop can be found [here](https://drive.google.com/drive/folders/15AbIeK20aPzfta24p-9fUSDcHu-uCw5v)

# Reading and processing the data

The four datasets can be retrieved individually as follows:

In [ ]:
# READING THE DATA
data_1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/DOH COVID Data Drop_ 20220618 - 04 Case Information_batch_0.csv')
data_2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/DOH COVID Data Drop_ 20220618 - 04 Case Information_batch_1.csv')
data_3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/DOH COVID Data Drop_ 20220618 - 04 Case Information_batch_2.csv')
data_4 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/DOH COVID Data Drop_ 20220618 - 04 Case Information_batch_3.csv')

print('Successfully loaded the data')

It would be ideal to combine all the datasets to a single `DataFrame`, hence performing `concat` would allow us to combine all of the datasets. However, the shapes of the datasets must first be verified to assure that all of the datasets have the same variables.

In [ ]:
print('Data 1 shape:', data_1.shape)
print('Data 2 shape:', data_2.shape)
print('Data 3 shape:', data_3.shape)
print('Data 4 shape:', data_4.shape)

We can also manually inspect the columns of the dataset by printing them out using `df.columns`

In [ ]:
data_1.columns

In [ ]:
data_2.columns

In [ ]:
data_3.columns

In [ ]:
data_4.columns

Upon inspection, we have now verified that the columns are the same per dataset. The said datasets can now be safely combined using `concat`

In [ ]:
doh_data = pd.concat([data_1, data_2, data_3, data_4])
doh_data.head()

## Data Dictionary

As taken from the DataDrop Field Metadata, the following features can be described as: 

| Variable          | Definition                                |
| ---------         | ----------------------------------------  |
| Casecode          | Random code assigned for labelling cases  |
| Age               | Age                                       |
| AgeGroup          | Five-year age group                       |
| Sex               | Sex                                       |
| DateSpecimen      | Date when specimen was collected          |
| DateResultRelease | Date of release result                    |
| DateRepConf       | Date publicly announced as confirmed case |
| DateDied          | Date died                                 |
| DateRecover       | Date Recovered                            |
| RemovalType       | Type of removal (recovery or death)       |
| Admitted          | Binary variable indicating that the patient has been taken to the hospital |
| RegionRes         | Region of residence |
| ProvRes           | Province of residence |
| CityMunRes        | City of residence |
| CityregPSGC      | Philippine Standard Geographic Code of regcipality or City of residence |
| BarangayRes       | Barangay of residence |
| BarangayPSGC     | Philippine Standard Geographic Code of regcipality or City of residence |
| HealthStatus      | Known current health status of patient (asymptomatic, mild, severe, critical, died, recovered) |
| Quarantined       | Identifies if the patient has ever been home quarantined |
| DateOnset         | Date of onset symptoms |
| Pregnanttab       | If the patient was pregnant at any point during the COVID-19 condition |

Basic information regarding the data can also be retrieved such as its descriptive statistics on applicable variables using `describe`

In [ ]:
doh_data.describe()

We would also be interested in identifying the different data type that is present on each variable using `dtypes`

In [ ]:
doh_data.dtypes

# Data Wrangling

As with any dataset, it would be ideal to identify errors in the values caused by faults in encoding the data. The following section aims to seek these kinds of errors in the data.

## Duplicates and Null Values

In [ ]:
doh_data.isna().sum()

CaseCode                   1
Age                     2906
AgeGroup                2906
Sex                       43
DateSpecimen          829920
DateResultRelease     831379
DateRepConf                0
DateDied             3635482
DateRecover          3032685
RemovalType             4176
Admitted             2299493
RegionRes               3359
ProvRes                49881
CityMunRes             59045
CityMuniPSGC           58931
BarangayRes           316142
BarangayPSGC          316000
HealthStatus               0
Quarantined              146
DateOnset            2327362
Pregnanttab          1810717
ValidationStatus      513669
dtype: int64

As indicated by the cell above, most of the variables have a missing value pertaining to the case's information. However, it would be prudent to refrain from dropping these cases given the context of the dataset. Certain relationships between variables could thus be explored which makes use of the available values per variable.

Duplicated data, however, can be safely dropped as these cases wold affect the accuracy of certain statistical information regarding the dataset. 

In [ ]:
doh_data.duplicated().sum()

0

Since the returned value for the number of duplicated rows in the dataset is zero, we can assume that there are no redundant information regarding the dataset. If we tried to check on the duplicated values for the `CaseCode`, however, a certain value will be returned by the `duplicated()` function. We can disregard this value since it was stated by the DOH that these values are not unique, such that they are randomly assigned to the patient's case.

In [ ]:
doh_data.duplicated(subset = ['CaseCode']).sum()

35595

# Geographical Data Analysis

Since we have identified that certain regions and cities in the country are more likely to have a greater number of cases compared to others, it would be helpful to create heatmaps to identify the COVID hotspots in the country.

In [ ]:
! pip install geopandas

Using the provided PSGC column in the `doh_data` dataset, we can pinpoint the geographical location of the cases in the country to create a heatmap. However, further datasets containing the latitude and longitude of a certain PSGC code must be collected. The PSGC shape files were provided by **altcoder** which was placed in the data folder for this analysis.

Philippines PSGC Administrative Boundaries Shapefiles: https://github.com/altcoder/philippines-psgc-shapefiles

In [ ]:
import geopandas as gpd

### Accessing and Reading the datasets

In [ ]:
# READING THE SHAPE DATASETS IN CSV FORMAT
baran_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Shape Files/Barangays.csv')
baran_data.columns

In [ ]:
# READING THE SHAPE DATASETS IN CSV FORMAT
baran_datashape = gpd.read_file('/content/drive/My Drive/Colab Notebooks/data/Shape Files/BarangaysMin/BarangaysMin.shp')
baran_datashape.head()

In [ ]:
doh_data.columns

In [ ]:
doh_data[['CityMuniPSGC', 'BarangayPSGC']]

In [ ]:
print('Number of Missing Municipal PSGC Data: ', doh_data['CityMuniPSGC'].isna().sum())
print('Number of Missing Barangay PSGC Data: ', doh_data['BarangayPSGC'].isna().sum())

Since there is a significant number of null values for the barangay and municipal columns for each recorded observations, it would be ideal not to drop these rows. A separate observation can be implemented for these values regarding their geographical data. In preparation for this, the scope of the exploration in this part will be limited to a single region.

### Narrowing the Scope of our exploration

To narrow the scope of the DataFrame to what is needed for this part of the Exploratory Data Analysis, a new DataFrame object will be created based on the filtering done to baran_datashape limite the data to only ones from the NCR Region. This new DataFrame will be called NCR_datashape.

In [ ]:
# Narrow dataframe scope to the NCR Region
NCR_datashape = baran_datashape[baran_datashape['ADM1_EN'] == 'National Capital Region']
NCR_datashape = NCR_datashape.rename(columns = {'ADM4_EN' : 'Barangay', 'ADM4_PCODE' : 'BarangayPSGC', 'ADM3_EN' : 'city', 'ADM3_PCODE' : 'city_psgc'})
NCR_datashape.sort_values(by='Barangay', ascending=True)
NCR_datashape.head()

Next, the same process will be done to the data on the COVID-19 cases, with only select columns to be used later. This new DatFrame will be called NCR_case_data

In [ ]:
NCR_case_data = doh_data[doh_data['ProvRes'] == 'NCR']
NCR_case_data = NCR_case_data[['CaseCode', 'DateRepConf', 'DateDied', 'DateRecover', 'CityMunRes', 'CityMuniPSGC', 'BarangayRes', 'BarangayPSGC', 'HealthStatus']]
NCR_case_data.head()

After narrowing the scope to just the NCR region, a heat map will be created showing the prevalence of COVID-19 in a the cities/municipalities of the region.

### Data Cleaning and Processing for plotting the HeatMap

To prepare for creating the HeatMap, the data will need to be processed to remove any further unnecessary data and create a final DataFrame object to be used as data to plot the HeatMap.

First, a new column of 1's will be appended to the NCR_case_data DataFrame


In [ ]:
NCR_case_data['CaseCode_n'] = 1
NCR_case_data.head()

Next, a new DataFrame containing the Baranagy PSGC code, Barangay Number, and total number of cases per Barangay (Case_counts), called NCR_case_test will be created from NCR_case_data.

In [ ]:
NCR_case_test = NCR_case_data.groupby(['BarangayPSGC', 'BarangayRes'], as_index = True)['CaseCode_n'].sum()
NCR_case_test = NCR_case_test.reset_index(name = 'Case_Counts')
NCR_case_test.head()

Next, another DataFrame containing the geographical location data (Coordinates, Perimeter, and Area) will be created using the BarangayPSGC and each grouped with an instance of Coordinates, Perimeter, and Area of the Barangay the PSGC code refers to.

In [ ]:
case_geometry = pd.DataFrame(columns = ['geometry'])
i = 0
for x in NCR_case_test['BarangayPSGC']:
  for y, z, a, b in zip(NCR_datashape['BarangayPSGC'], NCR_datashape['geometry'], NCR_datashape['PERIMETER'], NCR_datashape['AREA']):
    if x == y:
      case_geometry = case_geometry.append({'BarangayPSGC' : x,'geometry' : z, 'PERIMETER' : a, 'AREA' : b}, ignore_index = True)
print(case_geometry.columns);
print(case_geometry.shape)

In [ ]:
case_geometry = case_geometry.rename(columns = {'ADM4_PCODE' : 'BarangayPSGC'})
case_geometry.head()

Finally, the two DataFrames that were created earlier are merged into a third, final DataFrame for use in the plotting of the heatmap.

In [ ]:
final_case_test = NCR_case_test.merge(case_geometry, how = 'inner', on = 'BarangayPSGC')
final_case_test.head()

### Plotting the HeatMap

Finally, this last part is for plotting the heatmap. It begins with merging the final_case_test DataFrame with the NCR_datashape Dataframe, which means the data on the Baranagays and the number of cases per Barangay will be assigned is geographical location and coordinates based on the Barangay PSGC code assigned to it.

In [ ]:
Case_counts = NCR_datashape.merge(final_case_test, on = 'BarangayPSGC')
Case_counts = Case_counts.drop(columns = ['fid',	'cat',	'cat__1',	'cat_', 'ADM1_PCODE',	'ADM2_PCODE',	'ADM2_EN',	'ADM1_EN',	'ADM_ID',	'UPDATED', 'geometry_y', 'PERIMETER_y', 'AREA_y', 'BarangayRes'])
Case_counts = Case_counts.rename(columns = {'geometry_x' : 'geometry', 'PERIMETER_x' : 'PERIMETER', 'AREA_x' : 'AREA'})
print(Case_counts.shape)
Case_counts.head()

Next, it is time to convert our Pandas Dataframe 'Case_counts' to a GeoPandas GeoDataframe to plot our heatmap.

In [ ]:
Case_counts = gpd.GeoDataFrame(Case_counts)
Case_counts.head()

Now, it's time to create the heatmap. The structure, appearance, and the values in the heatmap is dependent on the values in the GeoDataFrame used in plotting the heatmap, such as the column 'geometry', which is used to plot each barangay and the 'Case_counts' column to determine how dark each part is compared to the rest of the heatmap. The darker the color, the higher the value for that specifica part, which for this dataset means more cases.

In [ ]:
fig = plt.figure(figsize = (20, 30))
ax = fig.add_subplot(1, 1, 1)
Case_counts.plot(ax = ax, figsize = (20, 30), cmap=plt.cm.YlOrBr, legend = True, edgecolor = 'black', column = 'Case_Counts')

# Final Dataset

### Dataset Creation and Export to CSV file format

#### Cases Per Region


In [ ]:
from google.colab import files

In [ ]:
doh_data["RegionRes"].fillna("None Stated", inplace=True)
doh_data["CityMunRes"].fillna("None Stated", inplace=True)
regionList = doh_data["RegionRes"].unique()
print(regionList)
print(regionList.shape)

In [ ]:
for i in range(19):
    cases_per_region = doh_data.loc[doh_data['RegionRes'] == regionList[i]]
    filename = regionList[i] + ".csv"
    csvFile = cases_per_region.to_csv(filename)
    files.download(filename)

#### Cases per Municipality in NCR

In [ ]:
rValues = doh_data.loc[doh_data['RegionRes'] == 'NCR']
cityList = rValues["CityMunRes"].unique()
print(cityList)
print(cityList.shape)

In [ ]:
for i in range(18):
  cases_per_municipality = rValues.loc[rValues["CityMunRes"] == cityList[i]]
  filename = cityList[i] + ".csv"
  csvFile = cases_per_municipality.to_csv(filename)
  files.download(filename)

### Visualization

#### Regional Heatmap of the Philippines

##### Accessing Data

In [ ]:
reg_datashape = gpd.read_file('/content/drive/My Drive/Colab Notebooks/data/Shape Files/Region/Regions.shp')
reg_datashape.head()

##### Cleaning and Processing Data

In [ ]:
shapeRegList = reg_datashape['ADM1_EN'].unique()
sortShapeList = sorted(shapeRegList)
sortShapeList

In [ ]:
regional_case_data = doh_data[['CaseCode', 'DateRepConf', 'DateDied', 'DateRecover', 'RegionRes', 'HealthStatus']]
newRegList = regional_case_data['RegionRes'].unique()
sortRegList = sorted(newRegList)
sortRegList

In [ ]:
sortRegList.remove('None Stated')
sortRegList.remove('ROF')
for i in range(17):
  regional_case_data.replace(sortRegList[i], sortShapeList[i],inplace=True)
regional_case_data['RegionRes'].unique()

In [ ]:
regional_case_data['CaseCode_n'] = 1
regional_case_data.head()

In [ ]:
region_case_test = regional_case_data.groupby(['RegionRes'], as_index = True)['CaseCode_n'].sum()
region_case_test = region_case_test.reset_index(name = 'Case_Counts')
region_case_test = region_case_test.rename(columns = {'RegionRes' : 'ADM1_EN'})
region_case_test.head()

In [ ]:
regional_geometry = pd.DataFrame(columns = ['geometry'])
i = 0
for x in region_case_test['ADM1_EN']:
  for y, z, a, b in zip(reg_datashape['ADM1_EN'], reg_datashape['geometry'], reg_datashape['PERIMETER'], reg_datashape['AREA']):
    if x == y:
      regional_geometry = regional_geometry.append({'ADM1_EN' : x,'geometry' : z, 'PERIMETER' : a, 'AREA' : b}, ignore_index = True)
regional_geometry.head()

In [ ]:
final_regcase_test = region_case_test.merge(regional_geometry, how = 'inner', on = 'ADM1_EN')
final_regcase_test.head()

##### Plotting the Heat Map

In [ ]:
RegCase_counts = reg_datashape.merge(final_regcase_test, on = 'ADM1_EN')
RegCase_counts = RegCase_counts.drop(columns = ['ADM_ID',	'UPDATED', 'geometry_y', 'PERIMETER_y', 'AREA_y'])
RegCase_counts = RegCase_counts.rename(columns = {'geometry_x' : 'geometry', 'PERIMETER_x' : 'PERIMETER', 'AREA_x' : 'AREA', 'ADM1_EN' : 'Region', 'ADM1_PCODE' : 'RegionPSGC'})

In [ ]:
RegCase_counts = gpd.GeoDataFrame(RegCase_counts)

In [ ]:
fig = plt.figure(figsize = (20, 30))
ax = fig.add_subplot(1, 1, 1)
RegCase_counts.plot(ax = ax, figsize = (20, 30), cmap=plt.cm.YlOrBr, legend = True, edgecolor = 'black', column = 'Case_Counts')

#### Municipal Heatmap of NCR

##### Accessing Data

In [ ]:
muni_datashape = gpd.read_file('/content/drive/My Drive/Colab Notebooks/data/Shape Files/MunicipalitiesManilaMergedMin/MunicipalitiesManilaMergedMin.shp')
muni_datashape.head()

##### Narrowing the Scope

In [ ]:
NCR_muni_datashape = muni_datashape[muni_datashape['ADM1_EN'] == 'National Capital Region']
NCR_muni_datashape = NCR_muni_datashape.rename(columns = {'ADM4_EN' : 'Barangay', 'ADM4_PCODE' : 'BarangayPSGC', 'ADM3_EN' : 'city', 'ADM3_PCODE' : 'city_psgc'})
NCR_muni_datashape.sort_values(by='city', ascending=True)
NCR_muni_datashape.head()

In [ ]:
NCR_municase_data = doh_data[doh_data['RegionRes'] == 'NCR']
NCR_municase_data = NCR_municase_data[['CaseCode', 'DateRepConf', 'DateDied', 'DateRecover', 'CityMunRes', 'CityMuniPSGC', 'BarangayRes', 'BarangayPSGC', 'HealthStatus']]
NCR_municase_data.head()

##### Cleaning and Processing Data

In [ ]:
NCR_municase_data['CaseCode_n'] = 1
NCR_municase_data.head()

In [ ]:
NCR_municase_test = NCR_municase_data.groupby(['CityMuniPSGC', 'CityMunRes'], as_index = True)['CaseCode_n'].sum()
NCR_municase_test = NCR_municase_test.reset_index(name = 'Case_Counts')
NCR_municase_test = NCR_municase_test.rename(columns = {'CityMuniPSGC' : 'city_psgc', 'CityMunRes' : 'city'})
NCR_municase_test.head()

In [ ]:
municase_geometry = pd.DataFrame(columns = ['geometry'])
i = 0
for x in NCR_municase_test['city_psgc']:
  for y, z, a, b in zip(NCR_muni_datashape['city_psgc'], NCR_muni_datashape['geometry'], NCR_muni_datashape['PERIMETER'], NCR_muni_datashape['AREA']):
    if x == y:
      municase_geometry = municase_geometry.append({'city_psgc' : x,'geometry' : z, 'PERIMETER' : a, 'AREA' : b}, ignore_index = True)
municase_geometry.head()

In [ ]:
final_municase_test = NCR_municase_test.merge(municase_geometry, how = 'inner', on = 'city_psgc')
final_municase_test.head()

##### Plotting the Heat Map

In [ ]:
MuniCase_counts = NCR_muni_datashape.merge(final_municase_test, on = 'city_psgc')
MuniCase_counts = MuniCase_counts.drop(columns = ['fid',	'cat',	'cat__1',	'cat_', 'ADM1_PCODE',	'ADM2_PCODE',	'ADM2_EN',	'ADM1_EN',	'ADM_ID',	'UPDATED', 'geometry_y', 'PERIMETER_y', 'AREA_y', 'city_y'])
MuniCase_counts = MuniCase_counts.rename(columns = {'geometry_x' : 'geometry', 'PERIMETER_x' : 'PERIMETER', 'AREA_x' : 'AREA', 'city_x' : 'City'})
print(MuniCase_counts.shape)
MuniCase_counts.head()

In [ ]:
MuniCase_counts = gpd.GeoDataFrame(MuniCase_counts)
MuniCase_counts.head()

In [ ]:
fig = plt.figure(figsize = (20, 30))
ax = fig.add_subplot(1, 1, 1)
MuniCase_counts.plot(ax = ax, figsize = (20, 30), cmap=plt.cm.YlOrBr, legend = True, edgecolor = 'black', column = 'Case_Counts')